In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv('/kaggle/input/playground-series-s3e3/train.csv')
test = pd.read_csv('/kaggle/input/playground-series-s3e3/test.csv')
train_add = pd.read_csv('/kaggle/input/ibm-hr-analytics-attrition-dataset/WA_Fn-UseC_-HR-Employee-Attrition.csv')

In [ ]:
train.head()

In [ ]:
train.info()

In [ ]:
train_add['Attrition']

In [ ]:
cat_cols = ['Age','BusinessTravel','Department','Education','EducationField','EmployeeCount','EnvironmentSatisfaction','Gender','JobInvolvement','JobLevel','JobRole','JobSatisfaction','MaritalStatus','NumCompaniesWorked','Over18','OverTime','PerformanceRating',
'RelationshipSatisfaction','StandardHours','StockOptionLevel','TrainingTimesLastYear','WorkLifeBalance','PercentSalaryHike','TotalWorkingYears','YearsAtCompany','YearsInCurrentRole','YearsWithCurrManager','YearsSinceLastPromotion']
for col in cat_cols:
    train[col] = train[col].astype('category')
    test[col] = test[col].astype('category')
train.drop(['id'],axis=1,inplace=True)   
test_id = test['id']
test.drop(['id'],axis=1,inplace=True)

train_add = train_add[train.columns]
for i,val in enumerate(train_add['Attrition']):
    if val=='No':
        train_add.iloc[i,-1] = 0
    else:
        train_add.iloc[i,-1] = 1

In [ ]:
train_add['Attrition']

In [ ]:
from category_encoders.cat_boost import CatBoostEncoder
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Activation,Add, Concatenate
from tensorflow.keras import Model, Input, Sequential
from sklearn.model_selection import train_test_split, KFold
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.regularizers import L2
from category_encoders.backward_difference import BackwardDifferenceEncoder
import optuna
from lightgbm import LGBMClassifier, LGBMRegressor
from sklearn.metrics import roc_auc_score, mean_squared_error



In [ ]:

X_train, X_cv, y_train, y_cv = train_test_split(train.iloc[:,:-1],train.iloc[:,-1],shuffle=True, random_state=1, test_size=0.2,stratify=train.iloc[:,-1].values)

encode = CatBoostEncoder(random_state=1)
X_train = encode.fit_transform(X_train,y_train)
X_cv = encode.transform(X_cv)
X_test = encode.transform(test)
X_train_add = encode.transform(train_add.loc[:,[col for col in train.columns if col!='Attrition']])

train_scaler = StandardScaler()
X_train[X_train.columns] = train_scaler.fit_transform(X_train)
X_cv[X_train.columns] = train_scaler.transform(X_cv)
X_train_add[X_train.columns] = train_scaler.transform(X_train_add)

test_scaler = StandardScaler()
X_test[X_train.columns] = test_scaler.fit_transform(X_test)


X_train = pd.concat([X_train,X_train_add],axis=0)
y_train = pd.concat([y_train,train_add['Attrition'].astype('bool')],axis=0)

In [ ]:
train_add['Attrition']

In [ ]:
#0.94
inputs = Input(shape=(33,))
X = inputs
X1, X2, X3, X4, X5, X6 = X, X, X, X, X, X
for i in [256,128,64]:
    X1 = Dense(units=i)(X1)
    X1 = BatchNormalization()(X1)
    X1 = Activation('relu')(X1)
    X1 = Dropout(0.8)(X1)
for i in [256,128,64]:
    X2 = Dense(units=i)(X2)
    X2 = BatchNormalization()(X2)
    X2 = Activation('elu')(X2)
    X2 = Dropout(0.8)(X2)
for i in [256,128,64]:
    X3 = Dense(units=i)(X3)
    X3 = BatchNormalization()(X3)
    X3 = Activation('tanh')(X3)
    X3 = Dropout(0.8)(X3)
for i in [256,128,64]:
    X4 = Dense(units=i)(X4)
    X4 = BatchNormalization()(X4)
    X4 = Activation('gelu')(X4)
    X4 = Dropout(0.8)(X4)
for i in [256,128,64]:
    X5 = Dense(units=i)(X5)
    X5 = BatchNormalization()(X5)
    X5 = Activation('selu')(X5)
    X5 = Dropout(0.5)(X5)
X1 = Dense(units=1)(X1)
X1 = BatchNormalization()(X1)
X1 = Activation('sigmoid')(X1)
X2 = Dense(units=1)(X2)
X2 = BatchNormalization()(X2)
X2 = Activation('sigmoid')(X2)
X3 = Dense(units=1)(X3)
X3 = BatchNormalization()(X3)
X3 = Activation('sigmoid')(X3)
X4 = Dense(units=1)(X4)
X4 = BatchNormalization()(X4)
X4 = Activation('sigmoid')(X4)
X5 = Dense(units=1)(X5)
X5 = BatchNormalization()(X5)
X5 = Activation('sigmoid')(X5)
outputs= Add()([X1,X2,X3,X4,X5])/5
model2 = Model(inputs=inputs, outputs=outputs)
model2.compile(loss=BinaryCrossentropy(),optimizer=Adam(),metrics=['AUC'])
model2.fit(X_train, y_train, validation_data=(X_cv,y_cv),epochs=100,batch_size=32)

In [ ]:
#0.94319
model = Sequential()
model.add(Input(shape=(33,)))
model.add(Dense(units=256))
model.add(BatchNormalization())
model.add(Dropout(0.8))
model.add(Activation('relu'))#ExponentialDecay(0.001,510,0.89)
model.add(Dense(units=128))
model.add(BatchNormalization())
model.add(Dropout(0.8))
model.add(Activation('relu'))
model.add(Dense(units=64))
model.add(BatchNormalization())
model.add(Dropout(0.8))
model.add(Activation('relu'))
model.add(Dense(units=1))
model.add(BatchNormalization())
model.add(Activation('sigmoid'))
model.compile(loss=BinaryCrossentropy(), optimizer= Adam(), metrics=['AUC'])
model.fit(X_train,y_train,epochs=120,batch_size=32,validation_data=(X_cv,y_cv))

kf = KFold(n_splits=2)

def objective(trial):
    params={
          'max_depth':trial.suggest_int('max_depth',-1,20),
           'min_data_in_leaf':trial.suggest_int('min_data_in_leaf',0,1000),
           'bagging_fraction':trial.suggest_float('bagging_fraction',0.0,1),
          'pos_bagging_fraction':trial.suggest_float('pos_bagging_fraction',0.0,1),
          'neg_bagging_fraction':trial.suggest_float('neg_bagging_fraction',0.0,1),
           'bagging_freq':trial.suggest_int('bagging_freq',0,1000),
           'lambda_l1':trial.suggest_loguniform('lambda_l1',1e-05,100),
           'lambda_l2':trial.suggest_loguniform('lambda_l2',1e-05,100),
          'num_leaves':trial.suggest_int('num_leaves',10,1000),
            'metrics':'mse',
            'max_bin':trial.suggest_int('max_bin',5,2000),
            'learning_rate':trial.suggest_loguniform('learning_rate',0.0001,0.1),
            'boosting':trial.suggest_categorical('boosting',['dart','gbdt']),
           'min_child_samples': trial.suggest_int('min_child_samples', 3, 2000),
             'cat_smooth':trial.suggest_int('cat_smooth', 1, 100)}
            
    score_list=[]
    for fold, (train_indx,cv_indx) in enumerate(kf.split(X_train,y_train)):
        X_tr = X_train.iloc[train_indx]
        y_tr = y_train.iloc[train_indx]
        X_cv = X_train.iloc[cv_indx]
        y_cv = y_train.iloc[cv_indx]
        
        
        model = LGBMRegressor(**params)
        model.fit(X_tr, y_tr,eval_set=(X_cv,y_cv), early_stopping_rounds=200,verbose=0)
        y_pred = model.predict(X_cv)
        score = mean_squared_error(y_cv,y_pred)
        score_list.append(score)
    return sum(score_list)/len(score_list)

study = optuna.create_study(direction='minimize')
study.optimize(objective,n_trials=1000)

        

In [ ]:

inputs = Input(shape=(33,))
X = inputs
X1, X2, X3, X4, X5 = X, X, X, X, X
for i in [500,400,300,200,100]:
    X1 = Dense(units=i)(X1)
    X1 = BatchNormalization()(X1)
    X1 = Activation('relu')(X1)
    X1 = Dropout(0.8)(X1)
for i in [500,400,300,200,100]:
    X2 = Dense(units=i)(X2)
    X2 = BatchNormalization()(X2)
    X2 = Activation('selu')(X2)
    X2 = Dropout(0.8)(X2)
for i in [500,400,300,200,100]:
    X3 = Dense(units=i)(X3)
    X3 = BatchNormalization()(X3)
    X3 = Activation('tanh')(X3) 
    X3 = Dropout(0.8)(X3)
for i in [500,400,300,200,100]:
    X4 = Dense(units=i)(X4)
    X4 = BatchNormalization()(X4)
    X4 = Activation('elu')(X4)
    X4 = Dropout(0.8)(X4)
for i in [500,400,300,200,100]:
    X5 = Dense(units=i)(X5)
    X5 = BatchNormalization()(X5)
    X5 = Activation('gelu')(X5)
    X5 = Dropout(0.8)(X5)
X = Concatenate(axis=-1)([X1,X2,X3,X4,X5])
X = Dense(units=5)(X)
X = BatchNormalization()(X)
X = Dense(units=1)(X)
X = BatchNormalization()(X)
outputs = Activation('sigmoid')(X)
model4 = Model(inputs=inputs, outputs=outputs)
model4.compile(loss=BinaryCrossentropy(), optimizer= Adam(), metrics=['AUC'])
model4.fit(X_train,y_train,epochs=100,batch_size=32,validation_data=(X_cv,y_cv))

In [ ]:
preds = ( model4.predict(X_test).reshape(-1,)+model2.predict(X_test).reshape(-1,)+model.predict(X_test).reshape(-1,))*0.333333333333333333

In [ ]:
submission = pd.DataFrame({'id':test_id,'Attrition':preds})
submission.to_csv('submission.csv',index=False)